In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
from torch.nn.functional import one_hot

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
import torch.nn as nn
import torch.nn.functional as F



class summary_layer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(summary_layer, self).__init__()

        self.beta_I = torch.eye(in_dim,in_dim).to(device)
        self.beta_func1 = nn.Linear(in_dim,out_dim).to(device)
        
    def forward(self, x):

        beta = self.beta_func1(self.beta_I)
        
        x1 = x@beta
        x2 = x@beta@(beta.T@beta)

        return x1,x2

    
class ConceptNet(nn.Module):
    def __init__(self,num_classes):
        super(ConceptNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.summary_layer1 = summary_layer(64*5*5, 120)
        self.output = nn.Linear(120,10)

    def forward(self, x):
        x = self.pool((self.conv1(x)))
        x = self.pool((self.conv2(x)))
        x1,x2 = self.summary_layer1(torch.flatten(x,1))
        return self.output(x1),self.output(x2)
        return x

    

def ConceptNet_simple(num_classes):
    return ConceptNet(num_classes)
    

In [6]:
import torch.optim as optim

net = ConceptNet_simple(10)
net.to(device)

mseloss = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [7]:
import time

start = time.time()

results = []

for epoch in range(15):  # loop over the dataset multiple times
    count=0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        count=i
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs1, outputs2 = net(inputs)
        
        loss = criterion(outputs1, labels)+mseloss(outputs1, outputs2)+criterion(outputs2,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # print statistics

    print(f'[{epoch + 1}, {count + 1:5d}] loss: {running_loss  :.10f}')
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs1,outputs2 = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs1.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')   
    
    
    end = time.time()
    print(end - start)
    results += [[epoch,end-start,running_loss,100*correct/total]]
    running_loss = 0.0
print('Finished Training')

end = time.time()
print(end - start)

[1,  1563] loss: 8223.2205705643
Accuracy of the network on the 10000 test images: 39.34 %
18.46623468399048
[2,  1563] loss: 5158.8486618996
Accuracy of the network on the 10000 test images: 45.34 %
36.03906512260437
[3,  1563] loss: 4699.1728965044
Accuracy of the network on the 10000 test images: 50.27 %
53.28981161117554
[4,  1563] loss: 4419.6741008759
Accuracy of the network on the 10000 test images: 53.48 %
70.3399658203125
[5,  1563] loss: 4159.0516985655
Accuracy of the network on the 10000 test images: 54.52 %
87.48525500297546
[6,  1563] loss: 3932.7443634272
Accuracy of the network on the 10000 test images: 58.15 %
104.44660186767578
[7,  1563] loss: 3739.5332360268
Accuracy of the network on the 10000 test images: 60.87 %
121.24568295478821
[8,  1563] loss: 3598.1824537516
Accuracy of the network on the 10000 test images: 60.99 %
138.2134325504303
[9,  1563] loss: 3483.8859317303
Accuracy of the network on the 10000 test images: 62.73 %
155.27910590171814
[10,  1563] loss:

In [ ]:
main_results = np.asarray(results)

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(15,5),dpi=120)
fig.suptitle('Base CNN with Convolution and a Summary Layer')
ax1.plot(main_results[:,0], main_results[:,1])
ax1.set_title("Runtime")
ax2.plot(main_results[:,0], main_results[:,2])
ax2.set_title("Loss metric")
ax2.set_xlabel('Epochs', fontsize=16)
ax3.plot(main_results[:,0], main_results[:,3])
ax3.set_title("Accuracy on test")

np.savetxt("../results_CNN_summary.csv", main_results, delimiter=",")